使用least_squares库中LM算法

## 步骤1：计算T0_3变换矩阵

In [ ]:
import numpy as np
from scipy.optimize import least_squares

def compute_T0_3(x, y, z, nx, ny, nz, ox, oy, oz, ax, ay, az, theta4, theta5, theta6, dh_params):
    """
    计算中间变换矩阵T0_3
    
    参数:
    x, y, z: TCP位置坐标
    nx, ny, nz, ox, oy, oz, ax, ay, az: TCP姿态矩阵元素
    theta4, theta5, theta6: 已知的关节4,5,6角度(弧度)
    dh_params: 机器人的DH参数字典
    
    返回:
    T0_3: 从基座标系到关节3坐标系的变换矩阵
    """
    # 1. 构建总体变换矩阵T
    T = np.array([
        [nx, ox, ax, x],
        [ny, oy, ay, y],
        [nz, oz, az, z],
        [0, 0, 0, 1]
    ])
    
    # 2. 计算T3_6 (关节3到关节6的变换)
    # 提取DH参数
    a3 = dh_params.get('a3', 0)
    a4 = dh_params.get('a4', 0)
    a5 = dh_params.get('a5', 0)
    d4 = dh_params.get('d4', 0)
    d5 = dh_params.get('d5', 0)
    d6 = dh_params.get('d6', 0)
    alpha3 = dh_params.get('alpha3', 0)
    alpha4 = dh_params.get('alpha4', 0)
    alpha5 = dh_params.get('alpha5', 0)
    
    # 计算T3_4, T4_5, T5_6
    T3_4 = dh_transform(a3, alpha3, d4, theta4)
    T4_5 = dh_transform(a4, alpha4, d5, theta5)
    T5_6 = dh_transform(a5, alpha5, d6, theta6+180*np.pi/180)  # 注意theta6需要加上180度
    
    # 计算T3_6
    T3_6 = np.dot(np.dot(T3_4, T4_5), T5_6)
    
    # 3. 计算T0_3 = T * (T3_6)^-1
    T3_6_inv = np.linalg.inv(T3_6)
    T0_3 = np.dot(T, T3_6_inv)
    print("T0_3:")
    print(T0_3)
    return T0_3

def dh_transform(a, alpha, d, theta):
    """
    计算DH变换矩阵
    
    参数:
    a: 连杆长度
    alpha: 连杆扭角(弧度)
    d: 连杆偏距
    theta: 关节角度(弧度)
    
    返回:
    T: 4x4的DH变换矩阵
    """
    # 构建DH变换矩阵
    ct = np.cos(theta)
    st = np.sin(theta)
    ca = np.cos(alpha)
    sa = np.sin(alpha)
    
    T = np.array([
        [ct, -st, 0, a],
        [st*ca, ct*ca, -sa, -d*sa],
        [st*sa, ct*sa, ca, d*ca],
        [0, 0, 0, 1]
    ])
    
    return T

## 步骤2：使用LM算法求解关节角度
现在我们有了T0_3矩阵，需要找到关节角度theta1, theta2, theta3，使得这些角度产生的变换矩阵尽可能接近计算出的T0_3。这是一个非线性最小二乘问题，非常适合使用Levenberg-Marquardt算法求解。

In [ ]:
def solve_joint_angles_lm_improved(T0_3, dh_params, initial_guess=None):
    """改进的LM算法求解关节角度"""
    # 提取DH参数
    a1 = dh_params.get('a1', 0)
    a2 = dh_params.get('a2', 0)
    d1 = dh_params.get('d1', 0)
    d2 = dh_params.get('d2', 0)
    d3 = dh_params.get('d3', 0)
    alpha1 = dh_params.get('alpha1', 0)
    alpha2 = dh_params.get('alpha2', 0)
    
    def improved_residuals(angles):
        theta1, theta2, theta3 = angles
        # 计算各关节变换矩阵
        T0_1 = dh_transform(0,0, d1, theta1)
        T1_2 = dh_transform(a1, alpha1, d2, theta2-90*np.pi/180)
        T2_3 = dh_transform(a2, alpha2, d3, theta3)
        
        # 计算预测的T0_3
        T0_3_pred = np.dot(np.dot(T0_1, T1_2), T2_3)
        
        # 计算位置误差
        position_error = T0_3_pred[:3, 3] - T0_3[:3, 3]
        
        # 计算旋转误差 (使用对数映射)
        R_pred = T0_3_pred[:3, :3]
        R_target = T0_3[:3, :3]
        R_error = np.dot(R_pred, R_target.T)
        # 将旋转矩阵转换为轴角表示
        theta = np.arccos((np.trace(R_error) - 1) / 2)
        if theta < 1e-10:
            rotation_error = np.zeros(3)
        else:
            # 计算旋转误差 k*sin(θ) = (r32-r23, r13-r31, r21-r12)/2
            rotation_error = theta * np.array([
                # 计算旋转误差的x分量
                R_error[2, 1] - R_error[1, 2],
                # 计算旋转误差的y分量
                R_error[0, 2] - R_error[2, 0],
                # 计算旋转误差的z分量
                R_error[1, 0] - R_error[0, 1]
            ]) / (2 * np.sin(theta))

        # 设置位置和旋转的权重
        position_weight =1
        rotation_weight =600

        
        # 组合所有误差项
        residuals = np.concatenate([
            position_weight * position_error,
            rotation_weight * rotation_error
        ])
    
        return residuals 
    
    # 设置初始猜测值
    if initial_guess is None:
        initial_guess = [0.0, 0.0, 0.0]
    
    # 使用改进的LM算法参数
   # 调整 LM 求解器参数
    result = least_squares(
        improved_residuals, 
        initial_guess,
        method='lm',
        ftol=1e-14,      # 提高精度
        xtol=1e-14,
        gtol=1e-14,
        max_nfev=1000,   # 增加最大迭代次数
        f_scale=0.05,     # 调整损失函数尺度
        tr_solver='exact'# 使用精确求解器
    )
    
    
    # 获取计算结果
    theta1, theta2, theta3 = result.x
    
    # 规范化角度到[-pi, pi]范围域
    theta1 = ((theta1 + np.pi) % (2*np.pi)) - np.pi
    theta2 = ((theta2 + np.pi) % (2*np.pi)) - np.pi
    theta3 = ((theta3 + np.pi) % (2*np.pi)) - np.pi
    
    return theta1, theta2, theta3

## 步骤3：多初始值策略以避免局部最小值

In [ ]:
def solve_joint_angles_multi_init(T0_3, dh_params):
    """使用多个初始猜测值提高求解成功率"""
    # 设置多个初始猜测值
    initial_guesses = [
        [0.0, 0.0, 0.0],
        [np.pi/4, np.pi/4, np.pi/4],
        [-np.pi/4, np.pi/4, np.pi/4],
        [np.pi/4, -np.pi/4, np.pi/4],
        [np.pi/4, np.pi/4, -np.pi/4],
        [np.pi/2, 0.0, 0.0],
        [0.0, np.pi/2, 0.0],
        [0.0, 0.0, np.pi/2],
        [np.pi/3, np.pi/3, np.pi/3],
        [-np.pi/3, -np.pi/3, -np.pi/3],
        [np.pi/6, np.pi/6, np.pi/6],
        [-np.pi/6, -np.pi/6, -np.pi/6]
    ]
    
    best_solution = None
    min_error = float('inf')
    
    for guess in initial_guesses:
        try:
            # 尝试求解
            theta1, theta2, theta3 = solve_joint_angles_lm_improved(T0_3, dh_params, guess)
            
            # 验证解的质量
            error = compute_solution_error(theta1, theta2, theta3, T0_3, dh_params)
            
            if error < min_error:
                min_error = error
                best_solution = (theta1, theta2, theta3)
        except:
            continue
    
    if best_solution is not None:
        return best_solution, min_error
    else:
        raise ValueError("无法找到有效解")

def compute_solution_error(theta1, theta2, theta3, target_T0_3, dh_params):
    """计算解的误差"""
    # 提取DH参数
    a1 = dh_params.get('a1', 0)
    a2 = dh_params.get('a2', 0)
    d1 = dh_params.get('d1', 0)
    d2 = dh_params.get('d2', 0)
    d3 = dh_params.get('d3', 0)
    alpha1 = dh_params.get('alpha1', 0)
    alpha2 = dh_params.get('alpha2', 0)
    
    # 计算各关节变换矩阵

    T0_1 = dh_transform(0,0, d1, theta1)
    T1_2 = dh_transform(a1, alpha1, d2, theta2-np.pi/2)  # 第二个关节有90度的偏移
    T2_3 = dh_transform(a2, alpha2, d3, theta3)  # 
    
    # 计算预测的T0_3
    T0_3_pred = np.dot(np.dot(T0_1, T1_2), T2_3)
    
    # 计算误差
    error_matrix = T0_3_pred - target_T0_3
    
    # 计算Frobenius范数作为总体误差
    return np.linalg.norm(error_matrix[:3, :])

## 完整的求解

In [ ]:
def solve_inverse_kinematics_lm(x, y, z, nx, ny, nz, ox, oy, oz, ax, ay, az, 
                              theta4, theta5, theta6, dh_params,initial_guess=None):
    """
    使用LM方法求解机器人反向运动学
    
    参数:
    x, y, z: TCP位置坐标
    nx, ny, nz, ox, oy, oz, ax, ay, az: TCP姿态矩阵元素
    theta4, theta5, theta6: 已知的关节4,5,6角度(弧度)
    dh_params: 机器人的DH参数字典
    
    返回:
    theta1, theta2, theta3: 计算得到的关节1、2、3的角度(弧度)
    error: 解的误差
    """
    # 1. 计算T0_3
    T0_3 = compute_T0_3(x, y, z, nx, ny, nz, ox, oy, oz, ax, ay, az, 
                        theta4, theta5, theta6, dh_params)
    
    # 2. 使用多初始值策略求解关节角度
    theta1, theta2, theta3= solve_joint_angles_lm_improved(T0_3, dh_params,initial_guess=initial_guess)
    
    # 3. 打印结果和误差
    print(f"求解结果:")
    print(f"关节1角度: {np.degrees(theta1):.2f}°")
    print(f"关节2角度: {np.degrees(theta2):.2f}°")
    print(f"关节3角度: {np.degrees(theta3):.2f}°")
    
    
    return theta1, theta2, theta3

In [ ]:

from scipy.spatial.transform import Rotation 

x, y, z = 913.612,745.315,633.829  # 位置(mm)  
quat = [0.8465,-0.3332,0.3434,0.2335] #[x y z w]  # 四元数
# 已知的关节4,5,6角度(弧度)  
theta4 = np.radians(-50.023)  
theta5 = np.radians(77.220)  
theta6 = np.radians(-75.153) 

x, y, z = 1078.478,268.266,565.881  # 位置(mm)  
quat = [-0.5804,0.7087,-0.3541,-0.1885] #[x y z w]  # 四元数
# 已知的关节4,5,6角度(弧度)  
theta4 = np.radians(-51.241)  
theta5 = np.radians(69.480)  
theta6 = np.radians(-34.404) 

x, y, z = 915.249, 608.436, 233.411
quat = [0.3604,0.8224,0.3919,-0.2006]
# 已知的关节4,5,6角度(弧度)  
theta4 = np.radians(90)  
theta5 = np.radians(45)  
theta6 = np.radians(0) 
# 创建旋转对象
rot = Rotation.from_quat(quat)

# 获取旋转矩阵
rotation_matrix = rot.as_matrix() 

# 提取姿态矩阵的各列向量  
nx,ny,nz = rotation_matrix[:,0]  # x轴方向向量
ox,oy,oz = rotation_matrix[:,1]  # y轴方向向量  
ax,ay,az = rotation_matrix[:,2] 

# 设置机器人DH参数(工业机器人 NB12s-1214-5A)  
dh_params = {  
    'a0': 0,      # 连杆1长度(mm)
    'a1': 100,    # 连杆2长度(mm) 
    'a2': 680,    # 连杆3长度(mm)
    'a3': 50,     # 连杆4长度(mm)
    'a4': 0,      # 连杆5长度(mm)
    'a5': 0,      # 连杆6长度(mm)
    'd1': 500,    # 关节1偏移(mm)
    'd2': 0,      # 关节2偏移(mm)
    'd3': 0,      # 关节3偏移(mm
    'd4': 660,    # 关节4偏移(mm)
    'd5': 0,      # 关节5偏移(mm)
    'd6': 98,     # 关节6偏移(mm)
    'alpha0': 0,          # 连杆0扭角(弧度)
    'alpha1': -np.pi/2,   # 连杆1扭角(弧度)
    'alpha2': 0,          # 连杆2扭角(弧度)
    'alpha3': -np.pi/2,   # 连杆3扭角(弧度)
    'alpha4': np.pi/2,    # 连杆4扭角(弧度)
    'alpha5': -np.pi/2,   # 连杆5扭角(弧度)
}

try:
    # 使用LM方法求解关节角度
    theta1, theta2, theta3 = solve_inverse_kinematics_lm(
        x, y, z, nx, ny, nz, ox, oy, oz, ax, ay, az, 
        theta4, theta5, theta6, dh_params
    )
    
    # 验证求解结果
    print("\n验证求解结果:")
    
    # 计算正向运动学
    # 计算变换矩阵T0_1, T1_2, T2_3, T3_4, T4_5, T5_6
    T0_1 = dh_transform(0, 0, dh_params['d1'], theta1)
    T1_2 = dh_transform(dh_params['a1'], dh_params['alpha1'], dh_params['d2'], theta2-np.pi/2)  # 第二个关节有90度的偏移
    T2_3 = dh_transform(dh_params['a2'], dh_params['alpha2'], dh_params['d3'], theta3)
    T3_4 = dh_transform(dh_params['a3'], dh_params['alpha3'], dh_params['d4'], theta4)
    T4_5 = dh_transform(0, dh_params['alpha4'], dh_params['d5'], theta5)
    T5_6 = dh_transform(0, dh_params['alpha5'], dh_params['d6'], theta6+np.pi)  # 注意theta6需要加上180度

    #计算T03
    T0_3 = np.dot(np.dot(T0_1, T1_2), T2_3)
    print("计算T0_3:")
    print(T0_3)
    # 计算总体变换矩阵
    T = np.dot(T0_1, np.dot(T1_2, np.dot(T2_3, np.dot(T3_4, np.dot(T4_5, T5_6)))))
    
    # 提取TCP位置和姿态
    x_verify, y_verify, z_verify = T[0:3, 3]
    
    # 计算位置误差
    position_error = np.sqrt((x-x_verify)**2 + (y-y_verify)**2 + (z-z_verify)**2)
    
    print(f"原始TCP位置: [{x}, {y}, {z}]")
    print(f"验证TCP位置: [{x_verify:.3f}, {y_verify:.3f}, {z_verify:.3f}]")
    print(f"位置误差: {position_error:.6f} mm")
    
    # 比较姿态矩阵
    rotation_error = np.linalg.norm(rotation_matrix - T[0:3, 0:3])
    print(f"姿态误差: {rotation_error:.6f}")
    
    # 打印所有关节角度
    print("\n完整关节角度:")
    print(f"关节1: {np.degrees(theta1):.3f}°")
    print(f"关节2: {np.degrees(theta2):.3f}°")
    print(f"关节3: {np.degrees(theta3):.3f}°")
    print(f"关节4: {np.degrees(theta4):.3f}°")
    print(f"关节5: {np.degrees(theta5):.3f}°")
    print(f"关节6: {np.degrees(theta6):.3f}°")
    
except Exception as e:
    print(f"求解过程出错: {e}")
    
    # 尝试使用不同的初始值或其他求解方法
    print("尝试使用备用求解方法...")
    # 这里可以添加备用求解方法，如几何法、解析法等

插补函数

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R
def slerp(q1, q2, t):
    """
    实现四元数球面线性插值
    
    参数:
        q1: 起始四元数 [x, y, z, w]
        q2: 终止四元数 [x, y, z, w]
        t: 插值参数 [0, 1]
    """
    # 确保四元数为单位四元数
    q1 = q1 / np.linalg.norm(q1)
    q2 = q2 / np.linalg.norm(q2)
    
    # 计算四元数点积
    dot = np.dot(q1, q2)
    
    # 如果点积为负，翻转其中一个四元数
    if dot < 0.0:
        q2 = -q2
        dot = -dot
    
    # 如果四元数非常接近，使用线性插值
    if dot > 0.9995:
        result = q1 + t * (q2 - q1)
        return result / np.linalg.norm(result)
    
    # 执行球面线性插值
    theta_0 = np.arccos(dot)
    sin_theta_0 = np.sin(theta_0)
    
    theta = theta_0 * t
    sin_theta = np.sin(theta)
    
    s0 = np.cos(theta) - dot * sin_theta / sin_theta_0
    s1 = sin_theta / sin_theta_0
    
    return s0 * q1 + s1 * q2

def cartesian_linear_interpolation(start_position, end_position, start_quat, end_quat, num_points):
    """
    笛卡尔空间线性插补
    
    参数:
        start_position: 起始TCP位置 [x, y, z]
        end_position: 终止TCP位置 [x, y, z]
        start_quat: 起始四元数 [x, y, z, w]
        end_quat: 终止四元数 [x, y, z, w]
        num_points: 插补点数量
        
    返回:
        positions: 插补后的位置列表
        quats: 插补后的四元数列表
    """
    positions = []
    quats = []
    
    # 确保四元数格式正确
    start_rot = start_quat
    end_rot = end_quat
    
    for i in range(num_points):
        t = i / (num_points - 1)
        
        # 线性插值位置
        pos = (1 - t) * np.array(start_position) + t * np.array(end_position)
        positions.append(pos)
        
        # 球面线性插值四元数
        interpolated_rot = slerp(start_rot, end_rot, t)
        quats.append(interpolated_rot)
    
    return positions, quats

def joint_linear_interpolation(start_joints, end_joints, num_points):
    """
    关节空间线性插补
    
    参数:
        start_joints: 起始关节角度 [j4, j5, j6]
        end_joints: 终止关节角度 [j4, j5, j6]
        num_points: 插补点数量
        
    返回:
        joints: 插补后的关节角度列表
    """
    joints = []
    start_joints = np.array(start_joints)
    end_joints = np.array(end_joints)
    
    for i in range(num_points):
        t = i / (num_points - 1)
        # 线性插值关节角度
        joint = (1 - t) * start_joints + t * end_joints
        joints.append(joint)
    
    return joints

def get_wrist_center(position, orientation, d6):
    """
    计算腕中心位置
    
    参数:
        position: TCP位置 [x, y, z]
        orientation: 四元数 [x, y, z, w]
        d6: 腕关节到TCP的偏移距离
        
    返回:
        wrist_center: 腕中心位置 [x, y, z]
    """
    # 将四元数转换为旋转矩阵
    rot = R.from_quat(orientation).as_matrix()
    
    # 计算工具z轴方向
    z_axis = rot[:, 2]
    
    # 腕中心位置 = TCP位置 - d6 * 工具z轴方向
    wrist_center = np.array(position) - d6 * z_axis
    
    return wrist_center

def interpolation_for_integration(start_position, end_position, start_quat, end_quat, 
                                  start_wrist_joints, end_wrist_joints, d6, num_points=50):
    """
    生成用于集成到您系统的插补数据
    
    参数:
        start_position: 起始TCP位置 [x, y, z]
        end_position: 终止TCP位置 [x, y, z]
        start_quat: 起始四元数 [x, y, z, w]
        end_quat: 终止四元数 [x, y, z, w]
        start_wrist_joints: 起始腕关节角度 [j4, j5, j6]
        end_wrist_joints: 终止腕关节角度 [j4, j5, j6]
        d6: 工具长度参数
        num_points: 插补点数量
        
    返回:
        插补结果字典
    """
    # 笛卡尔空间插补
    positions, quats = cartesian_linear_interpolation(
        start_position, end_position, start_quat, end_quat, num_points)
    
    # 关节空间插补
    wrist_joints = joint_linear_interpolation(
        start_wrist_joints, end_wrist_joints, num_points)
    
    # 计算腕中心位置
    wrist_centers = []
    for i in range(num_points):
        wc = get_wrist_center(positions[i], quats[i], d6)
        wrist_centers.append(wc)
    
    # 返回所有需要的数据
    return {
        'positions': np.array(positions),
        'quats': np.array(quats),
        'wrist_joints': np.array(wrist_joints),
        'wrist_centers': np.array(wrist_centers)
    }

轨迹求解函数

In [ ]:
def test_multiple_points(points_data):
    """测试多个路点的逆运动学求解
    
    参数:
        points_data: 包含多个路点信息的列表,每个路点是一个字典,包含:
            - position: [x, y, z]
            - quaternion: [qx, qy, qz, qw] 
            - wrist_joints: [theta4, theta5, theta6](degrees)
    """
    results = []
    poss = points_data['positions']
    quats = points_data['quats']
    wrists = [np.radians(a) for a in points_data['wrist_joints']]

    for i in range(50):
        print(f"\n========= 路点 {i+1} =========")
        pos = poss[i]
        quat = quats[i]
        wrist = wrists[i]
        print(f"位置: [{pos[0]:.3f}, {pos[1]:.3f}, {pos[2]:.3f}] mm")
        print(f"四元数: [{quat[0]:.4f}, {quat[1]:.4f}, {quat[2]:.4f}, {quat[3]:.4f}]")
        print(f"后三个关节角度: [{wrist[0]:.2f}, {wrist[1]:.2f}, {wrist[2]:.2f}] rad")
        
        # 创建旋转对象并获取旋转矩阵
        rot = Rotation.from_quat(quat)
        rotation_matrix = rot.as_matrix()
        
        # 提取姿态向量
        nx,ny,nz = rotation_matrix[:,0]
        ox,oy,oz = rotation_matrix[:,1]
        ax,ay,az = rotation_matrix[:,2]
        if len(results) == 0:
            initial_guess = [0.0, 0.0, 0.0]  # 默认初始猜测值
        else:
            initial_guess = [np.radians(a) for a in results[-1]['joint_angles'][:3]]
        print(f"初始值：{initial_guess}")   
        # 
        theta1_c, theta2_c, theta3_c= solve_inverse_kinematics_lm(
            pos[0], pos[1], pos[2], 
            nx, ny, nz, ox, oy, oz, ax, ay, az,
            wrist[0], wrist[1], wrist[2], 
            dh_params,initial_guess=initial_guess
        )
        
        joint_angles = [theta1_c, theta2_c, theta3_c, wrist[0], wrist[1], wrist[2]]
        # 验证求解结果
        print("\n验证求解结果:")
        
        # 计算正向运动学
        # 计算变换矩阵T0_1, T1_2, T2_3, T3_4, T4_5, T5_6
        T0_1 = dh_transform(0, 0, dh_params['d1'], joint_angles[0])
        T1_2 = dh_transform(dh_params['a1'], dh_params['alpha1'], dh_params['d2'], joint_angles[1]-np.pi/2)  # 第二个关节有90度的偏移
        T2_3 = dh_transform(dh_params['a2'], dh_params['alpha2'], dh_params['d3'], joint_angles[2])
        T3_4 = dh_transform(dh_params['a3'], dh_params['alpha3'], dh_params['d4'], joint_angles[3])
        T4_5 = dh_transform(0, dh_params['alpha4'], dh_params['d5'], joint_angles[4])
        T5_6 = dh_transform(0, dh_params['alpha5'], dh_params['d6'], joint_angles[5]+np.pi)  # 注意theta6需要加上180度

        # 计算总体变换矩阵
        T = np.dot(T0_1, np.dot(T1_2, np.dot(T2_3, np.dot(T3_4, np.dot(T4_5, T5_6)))))
        
        # 提取TCP位置和姿态
        x_verify, y_verify, z_verify = T[0:3, 3]
        
        # 计算位置误差
        position_error = np.sqrt((pos[0]-x_verify)**2 + (pos[1]-y_verify)**2 + (pos[2]-z_verify)**2)
        
        print(f"原始TCP位置: [{pos[0]}, {pos[1]}, {pos[2]}]")
        print(f"验证TCP位置: [{x_verify:.3f}, {y_verify:.3f}, {z_verify:.3f}]")
        print(f"位置误差: {position_error:.6f} mm")
        
        # 比较姿态矩阵
        rotation_error = np.linalg.norm(rotation_matrix - T[0:3, 0:3])
        print(f"姿态误差: {rotation_error:.6f}")

        results.append({
            'joint_angles': [np.degrees(a) for a in joint_angles],
            'position_error': position_error,
        })
        
        # 打印该路点的结果
        print("\n求解结果:")
        print(f"位置误差: {position_error:.4f} mm")
        print("关节角度(度):")
        for j, angle in enumerate(results[-1]['joint_angles']):
            print(f"J{j+1}: {angle:.3f}°")
            
    return results

# 定义测试路点
test_points = [
    {
        'position': [913.612, 745.315, 633.829],
        'quaternion': [0.8465, -0.3332, 0.3434, 0.2335],
        'wrist_angles': [-50.023, 77.220, -75.153]
    },
    {
        'position': [1078.478, 268.266, 565.881],
        'quaternion': [-0.5804, 0.7087, -0.3541, -0.1885],
        'wrist_angles': [-51.241, 69.480, -34.404]
    },
    {
        'position': [915.249, 608.436, 233.411],
        'quaternion': [0.3604, 0.8224, 0.3919, -0.2006],
        'wrist_angles': [90, 45, 0]
    }
]

start_pos=[1018.1418,40.4859,656.71229] # 起始位置
end_pos=[1018.1418,40.4859,1015.41402] # 目标位置
start_quat=[-0.0163,0.8205,0.0114,0.5713] # 起始四元数
end_quat=[-0.0163,0.8205,0.0114,0.5713] # 目标四元数
start_joint=[-0.00,-22.770,0.00]  # 起始关节角
end_joint=[-180,-6.578,180] # 目标关节角

points=interpolation_for_integration(start_pos,end_pos,start_quat,end_quat,start_joint,end_joint,d6=15)
# 运行测试
results = test_multiple_points(points)

# 统计分析
print("\n=== 统计分析 ===")
errors = [r['position_error'] for r in results]
print(f"平均位置误差: {np.mean(errors):.4f} mm")
print(f"最大位置误差: {np.max(errors):.4f} mm")
print(f"最小位置误差: {np.min(errors):.4f} mm")
print(f"误差标准差: {np.std(errors):.4f} mm")

绘图

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 绘制关节角度曲线
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))

# 提取所有关节角度数据
joint_angles = np.array([result['joint_angles'] for result in results])

# 绘制前3个关节角度（算法求解得到的）
time_points = np.arange(len(joint_angles))
ax1.plot(time_points, joint_angles[:, 0], 'r-', label='J1')
ax1.plot(time_points, joint_angles[:, 1], 'g-', label='J2')
ax1.plot(time_points, joint_angles[:, 2], 'b-', label='J3')
ax1.set_title('前3个关节角度变化曲线')
ax1.set_xlabel('插补点序号')
ax1.set_ylabel('关节角度 (度)')
ax1.grid(True)
ax1.legend()

# 绘制后3个关节角度（线性插值得到的）
ax2.plot(time_points, joint_angles[:, 3], 'r--', label='J4')
ax2.plot(time_points, joint_angles[:, 4], 'g--', label='J5')
ax2.plot(time_points, joint_angles[:, 5], 'b--', label='J6')
ax2.set_title('后3个关节角度变化曲线')
ax2.set_xlabel('插补点序号')
ax2.set_ylabel('关节角度 (度)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()

# 绘制位置误差随时间的变化
plt.figure(figsize=(10, 6))
plt.plot(time_points, [r['position_error'] for r in results], 'k-', label='位置误差')
plt.title('插补过程中的位置误差变化')
plt.xlabel('插补点序号')
plt.ylabel('位置误差 (mm)')
plt.grid(True)
plt.legend()
plt.show()